In [19]:
# How to run this notebook:
# 1. Install artiq (ARTIQ is installed in the nix shell)
# 2. Add neceddary packages (Jupyter, computational, etc.) to the flake.nix under the artiq folder
# 2. Run `nix develop` to activate the shell under the artiq folder
# 3. Run 'jupyter notebook' in the terminal under the artiq folder
# 4. Copy the url and use it to connect the notebook to the kernel
# 5. Run this notebook cells one by one

In [20]:
%matplotlib inline

import os
import logging
import time
import asyncio
import datetime
import glob
from pprint import pprint

import numpy as np
np.set_printoptions(precision=3)
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
import pandas as pd
import h5py

from sipyco.pc_rpc import (Client, AsyncioClient,
                           BestEffortClient, AutoTarget)
from artiq.master.databases import DeviceDB
from artiq.master.worker_db import DeviceManager

In [21]:
# Check if ipykernel is already installed   
# ! is used to run the command in the notebook cell
!jupyter kernelspec list

Available kernels:
  python3    /nix/store/1kfp16ilmj7d75r9qwjx78a89j2wlqf0-python3-3.12.6-env/lib/python3.12/site-packages/ipykernel/resources


In [22]:
# Install ipykernel (for the first time only)
# !python -m pip install ipykernel -U --user --force-reinstall

In [23]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /home/silviajojo/ARTIQ/my-artiq-project/artiq/run


In [24]:
print("Contents of current directory:", os.listdir())

Contents of current directory: []


In [25]:
# Create a new directory for the notebook
!mkdir -p ~/ARTIQ/my-artiq-project/artiq/run/
# Change the working directory to the new directory

In [17]:
# move to a location where we have our artiq setup
os.chdir(os.path.expanduser("~/ARTIQ/my-artiq-project/artiq/run"))

In [36]:
device_db_path = os.path.expanduser("~/ARTIQ/my-artiq-project/artiq/artiq/examples/no_hardware/device_db.py")
if os.path.exists(device_db_path):
    ddb = DeviceDB(device_db_path)
else:
    print(f"Error: device_db.py not found at {device_db_path}")

In [37]:
# First, let's check if we can even get the device
try:
    devmgr = DeviceManager(ddb)
    lda = devmgr.get("lda")
    print("LDA device reference obtained")
except Exception as e:
    print(f"Error getting LDA device: {e}")

# Then try to set attenuation with error handling
try:
    lda.set_attenuation(42)
    print("Attenuation set successfully")
except Exception as e:
    print(f"Error setting attenuation: {e}")

first connection attempt to ::1:3253[<class 'sipyco.pc_rpc.AutoTarget'>] failed, retrying in the background
Traceback (most recent call last):
  File "/nix/store/1kfp16ilmj7d75r9qwjx78a89j2wlqf0-python3-3.12.6-env/lib/python3.12/site-packages/sipyco/pc_rpc.py", line 323, in __init__
    self.__coninit(firstcon_timeout)
  File "/nix/store/1kfp16ilmj7d75r9qwjx78a89j2wlqf0-python3-3.12.6-env/lib/python3.12/site-packages/sipyco/pc_rpc.py", line 338, in __coninit
    self.__socket = socket.create_connection(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/nix/store/wfbjq35kxs6x83c3ncpfxdyl5gbhdx4h-python3-3.12.6/lib/python3.12/socket.py", line 865, in create_connection
    raise exceptions[0]
  File "/nix/store/wfbjq35kxs6x83c3ncpfxdyl5gbhdx4h-python3-3.12.6/lib/python3.12/socket.py", line 850, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused


LDA device reference obtained
Error setting attenuation: 


In [35]:
# 4. Verify that the attenuation was set correctly
assert lda.get_attenuation() == 42

# ... or we can wire it up ourselves if you know where it is
assert ddb.get("lda", resolve_alias=True)["host"] == "::1"
assert ddb.get("lda", resolve_alias=True)["port"] == 3253

# there are different Client types tailored to different use cases:

# synchronous
lda = Client("::1", 3253)
assert lda.get_attenuation() == 42

# asyncio
lda = AsyncioClient()
async def test_lda():
    await lda.connect_rpc("::1", 3253, AutoTarget)
    return await lda.get_attenuation()
assert asyncio.get_event_loop().run_until_complete(test_lda()) == 42

# best effort
lda = BestEffortClient("::1", 3253, AutoTarget)
assert lda.get_attenuation() == 42

first connection attempt to ::1:3253[<class 'sipyco.pc_rpc.AutoTarget'>] failed, retrying in the background
Traceback (most recent call last):
  File "/nix/store/1kfp16ilmj7d75r9qwjx78a89j2wlqf0-python3-3.12.6-env/lib/python3.12/site-packages/sipyco/pc_rpc.py", line 323, in __init__
    self.__coninit(firstcon_timeout)
  File "/nix/store/1kfp16ilmj7d75r9qwjx78a89j2wlqf0-python3-3.12.6-env/lib/python3.12/site-packages/sipyco/pc_rpc.py", line 338, in __coninit
    self.__socket = socket.create_connection(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/nix/store/wfbjq35kxs6x83c3ncpfxdyl5gbhdx4h-python3-3.12.6/lib/python3.12/socket.py", line 865, in create_connection
    raise exceptions[0]
  File "/nix/store/wfbjq35kxs6x83c3ncpfxdyl5gbhdx4h-python3-3.12.6/lib/python3.12/socket.py", line 850, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused


AttributeError: 

In [ ]:
# let's connect to the master

schedule, exps, datasets = [
    Client("::1", 3251, i) for i in
    "schedule experiment_db dataset_db".split()]

print("current schedule")
pprint(schedule.get_status())
print("experiments:")
pprint(exps.list_directory("repository"))

In [ ]:
# we can submit experiments to be run

expid = dict(
    file="repository/flopping_f_simulation.py",
    class_name="FloppingF",
    log_level=logging.WARNING,
    arguments=dict(
        F0=1500,
        noise_amplitude=.3,
    ),
)
if not schedule.get_status():
    rid = schedule.submit(pipeline_name="main", expid=expid,
        priority=0, due_date=None, flush=False)
print("current schedule")
pprint(schedule.get_status())

In [7]:
# wait for experiment to finish
# this can be written nicer by subscribing and reacting to scheduler changes
while rid in schedule.get_status():
    time.sleep(.1)

In [ ]:
# now that the experiment has completed, we can get the
# current value of the (live) dataset and plot it
# had we done this earlier, the dataset would have been incomplete
fig, ax = plt.subplots()
d = datasets.get("flopping_f_brightness")
ax.plot(d)
print("flopping_f:", datasets.get("flopping_freq"))

In [9]:
# this is how you would clear all pipelines
for i in schedule.get_status():
    schedule.delete(i)

In [ ]:
# we can easily find and use the data that was saved as part
# of the experiment

t = datetime.datetime.now()
f = os.path.join(
    "results", t.strftime("%Y-%m-%d"), #t.strftime("%H-%M"),
    "*", "{:09d}-FloppingF.h5".format(rid))

# we would usually like to use pandas but our data does not have
# the metadata pandas want
#d = pd.HDFStore(glob.glob(f)[0])

with h5py.File(glob.glob(f)[0]) as f:
    print("available datasets", list(f))
    assert np.allclose(f["datasets/flopping_f_brightness"], d)

In [ ]:
%%writefile repository/notebook_test.py

# we can also write experiments in the notebook and submit them
# we don't have submit-by-content yet (and there would be questions
# about other modules that would need to be imported) so we just export
# this cell and submit it by filename

from artiq.experiment import *

class Hello(EnvExperiment):
    def build(self):
        pass
    
    def run(self):
        print("Hello world!")

In [ ]:
expid = dict(
    file="repository/notebook_test.py",
    class_name="Hello",
    log_level=logging.WARNING,
    arguments=dict(),
)
rid = schedule.submit(pipeline_name="misc", expid=expid,
    priority=1, due_date=None, flush=False)
print(rid)
# on the master you should see the message.